# MusicXML Visualization

This notebook is phase two of this project, attempting to take what I've learned
about gathering data and visualize it.

## Prep

### Imports

In [17]:
# imports and constants
import music21
from pprint import pprint
from difficulty import *
from type21 import *
import warnings
import os
import numpy as np
import pandas as pd

### Parse MusicXML File

In [18]:
score = music21.converter.parse("../music/xml/musicalion/beethoven_egmont.musicxml")

## Extraction and Organization

In [19]:
currTempo = None
score_list = []

for part in score:
    try:
        instrument = part.getInstrument().instrumentName
    except:
        print(part)
        continue
    record_in_data_frame = True
    for element in part.recurse():
        # define items to be added to the data frame
        note_start = None
        note_end = None
        pitch = None
        volume = None
        tempo = None
        note_type = None

        if isChord(element):
            for note in element:
                note_start = note.offset
                note_end = note.offset + note.quarterLength
                pitch = note.pitch.ps
                volume = note.volume.realized
                tempo = currTempo
                note_type = type(note)
        elif isNote(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            pitch = element.pitch.ps
            volume = element.volume.realized
            note_type = type(element)
        elif type(element) == music21.tempo.MetronomeMark:
            record_in_data_frame = False
            currTempo = element.number
        elif isRest(element):
            note_start = element.offset
            note_end = element.offset + element.quarterLength
            note_type = type(element)
        if record_in_data_frame:
            score_list.append([note_start, note_end, pitch, volume, currTempo, note_type, instrument])


<music21.text.TextBox 'Seite #'>
<music21.layout.ScoreLayout>
<music21.layout.StaffGroup <music21.stream.PartStaff P1-Staff1><music21.stream.PartStaff P1-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P5-Staff1><music21.stream.PartStaff P5-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P10-Staff1><music21.stream.PartStaff P10-Staff2>>
<music21.layout.StaffGroup <music21.stream.PartStaff P1-Staff1><music21.stream.PartStaff P1-Staff2><music21.stream.Part 2 Oboi><music21.stream.Part 2 Clarinetti in B><music21.stream.Part 2 Fagotti>>
<music21.layout.StaffGroup <music21.stream.PartStaff P5-Staff1><music21.stream.PartStaff P5-Staff2><music21.stream.Part 2 Trombe in F><music21.stream.Part TImpani in F, c>>
<music21.layout.StaffGroup <music21.stream.PartStaff P8-Staff1><music21.stream.PartStaff P8-Staff2><music21.stream.Part Viola><music21.stream.PartStaff P10-Staf

In [20]:
df = pd.DataFrame(score_list, columns=['note_start', 'note_end', 'pitch', 'volume', 'tempo', 'note_type', 'instrument'])
df

,note_start,note_end,pitch,volume,tempo,note_type,instrument
0,None,None,NaN,NaN,None,None,Midi_74
1,None,None,NaN,NaN,None,None,Midi_74
2,None,None,NaN,NaN,None,None,Midi_74
3,None,None,NaN,NaN,None,None,Midi_74
4,None,None,NaN,NaN,None,None,Midi_74
...,...,...,...,...,...,...,...
25390,None,None,NaN,NaN,None,None,Midi_43
25391,0.0,1.0,41.0,1.0,None,<class 'music21.note.Note'>,Midi_43
25392,1.0,2.0,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_43
25393,2.0,4.0,NaN,NaN,None,<class 'music21.note.Rest'>,Midi_43


In [21]:
df1 = df.loc[(df['instrument'] == 'Midi_71') & (df['note_type'] == music21.note.Note)]
df1
# df1.plot(x="note_start", figsize=(20, 20), subplots=True)

,note_start,note_end,pitch,volume,tempo,note_type,instrument
6761,0.0,6.0,53.0,0.992124,None,<class 'music21.note.Note'>,Midi_71
6763,0.0,6.0,41.0,0.992124,None,<class 'music21.note.Note'>,Midi_71
6868,3.0,4.0,60.0,0.496062,None,<class 'music21.note.Note'>,Midi_71
6869,4.0,5.0,59.0,0.496062,None,<class 'music21.note.Note'>,Midi_71
6870,5.0,6.0,67.0,0.496062,None,<class 'music21.note.Note'>,Midi_71
...,...,...,...,...,...,...,...
9094,0.0,1.0,65.0,0.708660,None,<class 'music21.note.Note'>,Midi_71
9095,0.0,1.0,60.0,0.708660,None,<class 'music21.note.Note'>,Midi_71
9097,0.0,1.0,65.0,0.708660,None,<class 'music21.note.Note'>,Midi_71
9099,0.0,1.0,53.0,0.708660,None,<class 'music21.note.Note'>,Midi_71


In [28]:
# Group by instrument
df.groupby(['instrument']).mean()


,pitch,volume
instrument,,
Midi_43,64.184344,0.795118
Midi_48,44.453333,0.964899
Midi_57,67.011050,0.822566
Midi_61,71.895402,0.843392
Midi_69,79.078467,0.734630
Midi_71,57.358407,0.801058
Midi_72,74.188742,0.763427
Midi_74,86.683202,0.912939
